In [154]:
from LinkedBinaryTree import * 
import random
from game_tree import *


def build_set_tree_with_weights(tree):
    construct_tree(tree)

    visited = set()

    def assign_weights(node, win_on_receive_weight, momentum):
        if node is None or node in visited:
            return

        visited.add(node)

        if node.data == "WIN":
            node.right_weight = 1.0
            node.left_weight = 0.0
        elif node.data == "LOSE":
            node.right_weight = 0.0
            node.left_weight = 1.0
        else:
            node.right_weight = 0.5  # Default for winning
            node.left_weight = 0.5 # Default for losing

            if "40" in node.data and "0" not in node.data:
                node.right_weight += win_on_receive_weight

            if "15" in node.data or "30" in node.data:  # Example condition for momentum
                node.right_weight += momentum

            # Adjust weights if momentum is against the player
            if "0-" in node.data or "15-" in node.data or "30-0" in node.data:
                node.right_weight -= momentum
                node.left_weight += momentum

            # Normalize weights to ensure they sum to 1
            total_weight = node.right_weight + node.left_weight
            if total_weight > 0:
                node.right_weight /= total_weight
                node.left_weight /= total_weight

        assign_weights(node.left, win_on_receive_weight, momentum)
        assign_weights(node.right, win_on_receive_weight, momentum)

    assign_weights(tree.root, 0.1, 0.05)

def simulate_set(tree):
    player_games = 0
    opponent_games = 0

    while True:
        current_node = tree.root

        while current_node:
            if current_node.data == "WIN":
                player_games += 1
                break
            elif current_node.data == "LOSE":
                opponent_games += 1
                break

            random_value = random.random()
            if random_value < current_node.right_weight and current_node.right:
                current_node = current_node.right
            elif current_node.left:
                current_node = current_node.left

        if player_games >= 6 and player_games - opponent_games >= 2:
            return "Player wins the set", player_games, opponent_games
        if opponent_games >= 6 and opponent_games - player_games >= 2:
            return "Opponent wins the set", player_games, opponent_games

binary_tree = LinkedBinaryTree()
build_set_tree_with_weights(binary_tree)

result, player_games, opponent_games = simulate_set(binary_tree)
print(result)
print(f"Player Games: {player_games}, Opponent Games: {opponent_games}")


Opponent wins the set
Player Games: 4, Opponent Games: 6


In [155]:
def predict_set_winner_simulation(tree, match_data):
    player_games = int(match_data["p1_games"])
    opponent_games = int(match_data["p2_games"])
    player_score = int(match_data["p1_score"])
    opponent_score = int(match_data["p2_score"])

    while True:
        current_node = tree.root
        while current_node:
            random_value = random.random()
            if current_node.data == "WIN":
                player_score += 1
                if player_score == 4:  # Win game
                    player_games += 1
                    player_score = 0
                    opponent_score = 0
                break
            elif current_node.data == "LOSE":
                opponent_score += 1
                if opponent_score == 4:  # Lose game
                    opponent_games += 1
                    player_score = 0
                    opponent_score = 0
                break

            if random_value < current_node.right_weight and current_node.right:
                current_node = current_node.right
            elif current_node.left:
                current_node = current_node.left

        if player_games >= 6 and player_games - opponent_games >= 2:
            return "Player wins the set", player_games, opponent_games
        if opponent_games >= 6 and opponent_games - player_games >= 2:
            return "Opponent wins the set", player_games, opponent_games

binary_tree = LinkedBinaryTree()
build_set_tree_with_weights(binary_tree)


In [156]:
def build_set_tree_with_weights(tree, match_data):
    construct_tree(tree)

    visited = set()

    def assign_weights(node, win_on_receive_weight, momentum):
        if node is None or node in visited:
            return

        visited.add(node)

        # Determine who has momentum and advantage
        server = int(match_data["server"])
        point_victor = int(match_data["point_victor"])
        p1_break_pt = int(match_data["p1_break_pt"])
        p2_break_pt = int(match_data["p2_break_pt"])

        if node.data == "WIN":
            node.right_weight = 1.0
            node.left_weight = 0.0
        elif node.data == "LOSE":
            node.right_weight = 0.0
            node.left_weight = 1.0
        else:
            node.right_weight = 0.5
            node.left_weight = 0.5

            if server == 1 and p1_break_pt:
                node.right_weight += momentum
            elif server == 2 and p2_break_pt:
                node.left_weight += momentum

            if point_victor == 1:  # Player 1 has momentum
                node.right_weight += win_on_receive_weight
            elif point_victor == 2:  # Player 2 has momentum
                node.left_weight += win_on_receive_weight

            # Normalize weights to ensure they sum to 1
            total_weight = node.right_weight + node.left_weight
            if total_weight > 0:
                node.right_weight /= total_weight
                node.left_weight /= total_weight

        assign_weights(node.left, win_on_receive_weight, momentum)
        assign_weights(node.right, win_on_receive_weight, momentum)

    assign_weights(tree.root, 0.1, 0.05)

def find_start_node(tree, current_score):
    stack = [tree.root]
    while stack:
        node = stack.pop()
        if node.data == current_score:
            return node
        if node.right:
            stack.append(node.right)
        if node.left:
            stack.append(node.left)
    return None

def predict_set_winner_simulation(tree, match_data):
    player_games = int(match_data["p1_games"])
    opponent_games = int(match_data["p2_games"])
    player_score = match_data["p1_score"]
    opponent_score = match_data["p2_score"]
    current_score = f"{player_score}-{opponent_score}"

    current_node = find_start_node(tree, current_score)
    if current_node is None:
        raise ValueError(f"Score {current_score} not found in tree")

    while True:
        random_value = random.random()
        if current_node.data == "WIN":
            player_games += 1
            break
        elif current_node.data == "LOSE":
            opponent_games += 1
            break

        if random_value < current_node.right_weight and current_node.right:
            current_node = current_node.right
        elif current_node.left:
            current_node = current_node.left

        if player_games >= 6 and player_games - opponent_games >= 2:
            return "Player wins the set", player_games, opponent_games
        if opponent_games >= 6 and opponent_games - player_games >= 2:
            return "Opponent wins the set", player_games, opponent_games

binary_tree = LinkedBinaryTree()

In [157]:
import pandas as pd

# Load match data
match_data_df = pd.read_csv("m1.csv")

# Convert the first row of the DataFrame to a dictionary for prediction
match_data = match_data_df.iloc[0].to_dict()

# Predict winner by simulating forward
result, player_games, opponent_games = predict_set_winner_simulation(binary_tree, match_data)
print(result)
print(f"Player Games: {player_games}, Opponent Games: {opponent_games}")


AttributeError: 'NoneType' object has no attribute 'data'